In [3]:
import torch
import netCDF4 as nc
import xarray as xr
import sys
import lovely_tensors as lt
import torch.nn.functional as F
import numpy as np
from statsmodels.tsa.ar_model import AutoReg
from sklearn.metrics import mean_squared_error
from math import sqrt
import random

lt.monkey_patch()
sys.path.append('/app')

from Dataset import load_data

In [2]:
dataset = xr.open_mfdataset('/app/data/*.nc', combine='by_coords')
dataset

<xarray.Dataset>
Dimensions:            (time: 9295, lat: 337, lon: 384)
Coordinates:
  * time               (time) datetime64[ns] 1997-09-04 ... 2023-03-01
  * lat                (lat) float32 62.02 61.98 61.94 61.9 ... 48.1 48.06 48.02
  * lon                (lon) float32 -11.98 -11.94 -11.9 ... 3.896 3.938 3.979
Data variables:
    KD490              (time, lat, lon) float32 dask.array<chunksize=(830, 337, 384), meta=np.ndarray>
    ZSD_uncertainty    (time, lat, lon) float32 dask.array<chunksize=(830, 337, 384), meta=np.ndarray>
    KD490_uncertainty  (time, lat, lon) float32 dask.array<chunksize=(830, 337, 384), meta=np.ndarray>
    flags              (time, lat, lon) int8 dask.array<chunksize=(830, 337, 384), meta=np.ndarray>
    ZSD                (time, lat, lon) float32 dask.array<chunksize=(830, 337, 384), meta=np.ndarray>
Attributes: (12/92)
    lon_step:                        0.041666668
    keywords:                        EARTH SCIENCE > OCEANS > OCEAN OPTICS > ...
    cmems_product_id:                OCEANCOLOUR_GLO_BGC_L4_MY_009_104
    lat_step:                        0.041666668
    grid_mapping:                    Equirectangular
    title:                           cmems_obs-oc_glo_bgc-transp_my_l4-gapfre...
    ...                              ...
    westernmost_longitude:           -180.0
    geospatial_vertical_positive:    up
    date_created:                    2023-03-09T13:54:22Z
    nb_valid_bins:                   20803502
    pct_valid_bins:                  55.73640582133059
    History:                         Translated to CF-1.0 Conventions by Netc...

In [10]:
# validation date
start_date = '2020-01-01'
end_date = '2021-01-01'

In [15]:
subset = dataset.sel(time=slice(start_date, end_date))
subset

<xarray.Dataset>
Dimensions:            (time: 368, lat: 337, lon: 384)
Coordinates:
  * time               (time) datetime64[ns] 2020-01-01 ... 2021-01-01
  * lat                (lat) float32 62.02 61.98 61.94 61.9 ... 48.1 48.06 48.02
  * lon                (lon) float32 -11.98 -11.94 -11.9 ... 3.896 3.938 3.979
Data variables:
    KD490              (time, lat, lon) float32 dask.array<chunksize=(367, 337, 384), meta=np.ndarray>
    ZSD_uncertainty    (time, lat, lon) float32 dask.array<chunksize=(367, 337, 384), meta=np.ndarray>
    KD490_uncertainty  (time, lat, lon) float32 dask.array<chunksize=(367, 337, 384), meta=np.ndarray>
    flags              (time, lat, lon) int8 dask.array<chunksize=(367, 337, 384), meta=np.ndarray>
    ZSD                (time, lat, lon) float32 dask.array<chunksize=(367, 337, 384), meta=np.ndarray>
Attributes: (12/92)
    lon_step:                        0.041666668
    keywords:                        EARTH SCIENCE > OCEANS > OCEAN OPTICS > ...
    cmems_product_id:                OCEANCOLOUR_GLO_BGC_L4_MY_009_104
    lat_step:                        0.041666668
    grid_mapping:                    Equirectangular
    title:                           cmems_obs-oc_glo_bgc-transp_my_l4-gapfre...
    ...                              ...
    westernmost_longitude:           -180.0
    geospatial_vertical_positive:    up
    date_created:                    2023-03-09T13:54:22Z
    nb_valid_bins:                   20803502
    pct_valid_bins:                  55.73640582133059
    History:                         Translated to CF-1.0 Conventions by Netc...

In [40]:
def get_non_nan_periods(data, n_periods, period_length):
    non_nan_periods = []
    for _ in range(n_periods):
        while True:
            i, j = np.random.randint(0, data.shape[1]), np.random.randint(0, data.shape[2])
            random_start = np.random.randint(period_length, data.shape[0] - period_length)
            period = data[:random_start + period_length, i, j]
            if not np.isnan(period).any():
                non_nan_periods.append((random_start, i, j))
                break
    return non_nan_periods

In [86]:
def forecast_autoreg(series, lags):
    model = AutoReg(series[:-1], lags=lags)
    model_fit = model.fit()
    prediction = model_fit.predict(start=len(series), end=len(series))
    return prediction[0]

In [100]:
n_random_periods = 100_000  # adjust this value as needed
lags = 10
period_length = 12
random_periods = get_non_nan_periods(subset['ZSD'].values, n_random_periods, period_length)

In [101]:
rmse_list = []
for period in random_periods:
    start, lat, lon = period
    time_series = subset['ZSD'][:start + period_length, lat, lon].values
    actual = time_series[-1]
    forecast = forecast_autoreg(time_series[:-1], lags)
    rmse = sqrt(mean_squared_error([actual], [forecast]))
    rmse_list.append(rmse)

In [103]:
np.array(rmse_list).mean()

2.014501081324143

In [69]:
time_series.shape, time_series[:-1].shape

((342,), (341,))

In [80]:
model = AutoReg(time_series[:-1], lags=10).fit()
model.predict(time_series[:].shape[0], time_series.shape[0])

array([9.520133])

In [82]:
time_series[-10:]

array([ 8.304833,  9.543318, 10.304537,  9.740552,  9.728273, 12.051669,
       12.062932, 12.061241,  8.603299,  9.464794], dtype=float32)

In [62]:
len(time_series[-1:])

1

In [32]:
time_series[:-1].shape

(11,)